In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv('matches.csv')
data

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,...,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
1,2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,...,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City
2,3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,...,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City
3,4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,...,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City
4,6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,...,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384,38,2021-05-02,19:15,Premier League,Matchweek 34,Sun,Away,L,0.0,4.0,...,Match Report,NaN,8.0,1.0,17.4,0.0,0.0,0.0,2021,Sheffield United
1385,39,2021-05-08,15:00,Premier League,Matchweek 35,Sat,Home,L,0.0,2.0,...,Match Report,NaN,7.0,0.0,11.4,1.0,0.0,0.0,2021,Sheffield United
1386,40,2021-05-16,19:00,Premier League,Matchweek 36,Sun,Away,W,1.0,0.0,...,Match Report,NaN,10.0,3.0,17.0,0.0,0.0,0.0,2021,Sheffield United
1387,41,2021-05-19,18:00,Premier League,Matchweek 37,Wed,Away,L,0.0,1.0,...,Match Report,NaN,11.0,1.0,16.0,1.0,0.0,0.0,2021,Sheffield United


Data consists of: 
- 38 Matches each season
- 20 teams in the league per season, 
- 2 seasons total of including
- Relgated matches

In [3]:
data.shape

(1389, 28)

In [4]:
data["team"].value_counts()

team
Southampton                 72
Brighton and Hove Albion    72
Manchester United           72
West Ham United             72
Newcastle United            72
Burnley                     71
Leeds United                71
Crystal Palace              71
Manchester City             71
Wolverhampton Wanderers     71
Tottenham Hotspur           71
Arsenal                     71
Leicester City              70
Chelsea                     70
Aston Villa                 70
Everton                     70
Liverpool                   38
Fulham                      38
West Bromwich Albion        38
Sheffield United            38
Brentford                   34
Watford                     33
Norwich City                33
Name: count, dtype: int64

In [5]:
# Cleaning our Data
data['date'] = pd.to_datetime(data["date"]) # Turning dates into a datetime datatype
data["venue_code"] = data["venue"].astype("category").cat.codes # Converting home/away games into 1/0
data["opp_code"] = data["opponent"].astype("category").cat.codes # Giving each team a code
data['hour'] = data["time"].str.replace(":.+", "", regex=True).astype("int") # Time of day into hours
data["day_code"] = data["date"].dt.day_of_week # Finds day of the week
data['target'] = (data["result"] == "W").astype("int") # Wins/Loses, Draws -> 1/0



In [6]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)
train = data[data["date"] < '2022-01-01']
test = data[data["date"] > '2022-01-01'] # Predicting Future matches with old based on data from old matches

predictors = ['venue_code', 'opp_code', 'hour', 'day_code']
rf.fit(train[predictors], train['target'])


,n_estimators,50
,criterion,'gini'
,max_depth,None
,min_samples_split,10
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [7]:
preds = rf.predict(test[predictors])
accuarcy = accuracy_score(test["target"], preds)
accuarcy


0.6123188405797102

In [8]:
grouped_matches = data.groupby("team")


In [9]:
def rolling_averages(group, cols, new_cols):
    pass